In [5]:
import os
print(os.listdir("/Users/kiranguruv/Downloads/data_skintone"))


['.DS_Store', 'mid-dark', 'mid-light', 'light', 'dark']


In [4]:
import os

images = []
classes = []

base_path = "/Users/kiranguruv/Downloads/data_skintone"

for fold in os.listdir(base_path):
    path = os.path.join(base_path, fold)
    if os.path.isdir(path):  # ✅ Only process if it's a directory
        print(fold, len(os.listdir(path)))
        images.append(os.path.join(path, os.listdir(path)[0]))
        classes.append(fold)


mid-dark 10576
mid-light 6844
light 9769
dark 8640


In [6]:
for skintone in os.listdir(source_base):
    src_folder = os.path.join(source_base, skintone)
    if not os.path.isdir(src_folder):
        continue  # ✅ Skip .DS_Store or any stray files

    dst_folder = os.path.join(dest_base, skintone)
    os.makedirs(dst_folder, exist_ok=True)

    valid_count = 0

    print(f"Processing {skintone}...")

    for img_name in tqdm(os.listdir(src_folder)):
        if valid_count >= 1000:
            break

        img_path = os.path.join(src_folder, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(img_rgb)

        if result.multi_face_landmarks:
            landmarks = result.multi_face_landmarks[0].landmark
            h, w = img.shape[:2]

            if is_frontal_with_two_eyes(landmarks, w, h):
                shutil.copy(img_path, os.path.join(dst_folder, img_name))
                valid_count += 1

    print(f"✓ {valid_count} valid images saved for {skintone}\n")


Processing mid-dark...


 10%|███▊                                 | 1087/10576 [00:05<00:43, 216.03it/s]


✓ 1000 valid images saved for mid-dark

Processing mid-light...


 16%|█████▉                                | 1066/6844 [00:04<00:26, 216.11it/s]


✓ 1000 valid images saved for mid-light

Processing light...


 11%|████                                  | 1053/9769 [00:04<00:40, 217.88it/s]


✓ 1000 valid images saved for light

Processing dark...


 13%|████▊                                 | 1089/8640 [00:05<00:35, 215.53it/s]

✓ 1000 valid images saved for dark



In [2]:
import os

images = []
classes = []

base_path = "/Users/kiranguruv/Downloads/filtered_skintone"

for fold in os.listdir(base_path):
    path = os.path.join(base_path, fold)
    if os.path.isdir(path):  # ✅ Only process if it's a directory
        files = os.listdir(path)
        if len(files) == 0:
            print(f"{fold} is empty.")
            continue
        print(f"{fold} {len(files)}")
        images.append(os.path.join(path, files[0]))
        classes.append(fold)


.DS_Store is empty.
mid-dark 1000
mid-light 1000
light 1000
dark 1000
